In [3]:
import pandas as pd
import numpy as np
import uuid
import re
from datetime import datetime
from rdflib import Graph, Literal, Namespace, URIRef, BNode, RDF, RDFS
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS
from functions import *

In [4]:
COMMUNE = 'Gentilly'

In [5]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/gentilly/LANDMARKS.csv"
OUTPUT_PATH = ROOT + "data/rdf/"

landmarks = pd.read_csv(PATH,header=0)

In [6]:
from namespaces import *

# Create a new RDF graph
g = Graph()

#Define URIs
g.bind("landmark", landmarkuri)
g.bind("cad", cad)
g.bind("add", add)
g.bind("ltype", ltype)
g.bind("lrtype", lrtype)
g.bind("atype", atype)
g.bind("ctype", ctype)
g.bind("cad_ltype", cad_ltype)
g.bind("geofla", geofla)
g.bind("pwikidata", pwikidata)
g.bind("time", time)

# Iterate over each row in the DataFrame
for index, row in landmarks.iterrows():
    subject_uri = URIRef(landmarkuri + row['uuid'])
    g.add((subject_uri, RDF.type, add.Landmark))
    #add.isLandmarkType
    if row['add:LandmarkType'] in ['Section','Plot','Departement','Arrondissement','Canton']:
        g.add((subject_uri, add.isLandmarkType, URIRef(cad_ltype + f"{row['add:LandmarkType']}")))
    else:
        g.add((subject_uri, add.isLandmarkType, URIRef(ltype + f"{row['add:LandmarkType']}")))
    #rdfs:label
    if not pd.isnull(row['rdfs:label']):
        g.add((subject_uri, RDFS.label, Literal(row['rdfs:label'], datatype=XSD.string)))
    #dcterms:identifier
    if not pd.isnull(row['dcterms:identifier']):
        g.add((subject_uri, DCTERMS.identifier, Literal(row['dcterms:identifier'], datatype=XSD.string)))
    #geofla:numInsee
    if not pd.isnull(row['geofla:numInsee']):
        g.add((subject_uri, add.numInsee, Literal(int(row['geofla:numInsee']), datatype=XSD.string)))
    #wikidata:numEhess
    if not pd.isnull(row['wikidata:P8422']):
        g.add((subject_uri, pwikidata.P8422, Literal(int(row['wikidata:P8422']), datatype=XSD.string)))

    #Landmark Relations
    landmarkRelationNode = BNode()
    if not pd.isnull(row['Parent']):
        g.add((URIRef(landmarkRelationNode.n3()), RDF.type, add.LandmarkRelation))
        if not pd.isnull(row['Relation']):
            g.add((URIRef(landmarkRelationNode.n3()), add.isLandmarkRelationType, URIRef(lrtype + f"{row['Relation']}")))
        else:
            g.add((URIRef(landmarkRelationNode.n3()), add.isLandmarkRelationType, lrtype.Undefined))
        g.add((URIRef(landmarkRelationNode.n3()), add.locatum, subject_uri))
        parentuuid = find_uuid(landmarks, row['Parent'])
        g.add((URIRef(landmarkRelationNode.n3()), add.relatum, URIRef(landmarkuri + parentuuid)))

    #Landmark Appearance
    ## Change
    changeapp = BNode()
    g.add((URIRef(changeapp.n3()), RDF.type, add.Change))
    g.add((URIRef(changeapp.n3()),add.appliedTo,subject_uri))
    g.add((URIRef(changeapp.n3()),add.isChangeType,ctype.LandmarkAppearance))
    
    ## Event
    eventapp = BNode()
    g.add((URIRef(eventapp.n3()), RDF.type, add.Event))
    g.add((URIRef(changeapp.n3()),add.dependsOn,URIRef(eventapp.n3())))
    if not pd.isnull(row['hasBeginning']):
        time_ = BNode()
        g.add((URIRef(eventapp.n3()), add.hasTime, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar,time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision,time.Year))
        g.add((URIRef(time_.n3()), add.timeStamp, Literal(datetime.strptime(str(int(row['hasBeginning'])) + '-01-01', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))


    #Landmark Disappearance
    ## Change
    changedis = BNode()
    g.add((URIRef(changedis.n3()), RDF.type, add.Change))
    g.add((URIRef(changedis.n3()),add.appliedTo,subject_uri))
    g.add((URIRef(changedis.n3()),add.isChangeType,ctype.LandmarkDisappearance))
    
    ## Event
    eventdis= BNode()
    g.add((URIRef(eventdis.n3()), RDF.type, add.Event))
    g.add((URIRef(changedis.n3()),add.dependsOn,URIRef(eventdis.n3())))

    if not pd.isnull(row['hasEnd']):
        time_ = BNode()
        g.add((URIRef(eventdis.n3()), add.hasTime, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision, time.Year))
        g.add((URIRef(time_.n3()), add.timeStamp, Literal(datetime.strptime(str(int(row['hasEnd'])) + '-12-31', '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

print(g.serialize(format='turtle'))

#write the graph to a file
g.serialize(destination=f'{OUTPUT_PATH}/landmarks.ttl', format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad_ltype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/landmarkType/> .
@prefix ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix landmark: <http://rdf.geohistoricaldata.org/id/landmark/> .
@prefix lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/> .
@prefix ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/> .
@prefix pwikidata: <http://www.wikidata.org/wiki/Property:> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N0490959506f8423e9901fa92b8709e9c> a add:Change ;
    add:appliedTo landmark:8aeca660-ee93-4fef-b98a-026735807d82 ;
    add:dependsOn <_:N988e119804104f27ba539fbfe8d137a5> ;
    add:isChangeType ctype:LandmarkAppearance .

<_:N06108696942a447eb7bdcee20c052c7

<Graph identifier=Na2ceb108b6d848e19bb1641fa6ed78b0 (<class 'rdflib.graph.Graph'>)>